<a href="https://colab.research.google.com/github/Orshikhbayar/ml/blob/main/Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta, timezone
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
import ipywidgets as widgets
from IPython.display import display
import yfinance as yf

# Fetch available tickers from Yahoo Finance
tickers = ['BTC-USD', 'PXP-USD', 'XRP-USD', 'ETH-USD', 'XDC-USD', 'MCH-USD', 'XLM-USD', 'ETC-USD', 'REV-USD', 'DOGE-USD', '1INCH-USD', 'ADA-USD', 'VET-USD', 'VAL-USD', 'HBAR-USD', 'RSR-USD',
           'DFI-USD', 'LTC-USD', 'TRX-USD', 'NEO-USD', 'DGB-USD', 'BCH-USD', 'ZRX-USD', 'DOT-USD','PEPE-USD', 'DAI-USD', 'SOL-USD', 'VRA-USD', 'ENJ-USD', 'AVAX-USD', 'GALA-USD', 'ALGO-USD',
           'USDC-USD', 'OMG-USD', 'GRT-USD', 'BAT-USD', 'LINK-USD', 'SC-USD', 'COVN-USD', 'FTM-USD',
           'ZEC-USD', 'DASH-USD', 'WAVES-USD', 'UNI-USD', 'XTZ-USD', 'CRO-USD', 'ATOM-USD', 'CRV-USD',
           'QTUM-USD', 'CELO-USD', 'COMP-USD', 'ETHW-USD', 'AAVE-USD', 'MATIC-USD', 'SNX-USD', 'OCEAN-USD', 'SG-USD', '1ECO-USD', 'JASMY-USD', 'RVN-USD', 'DCR-USD',
           'FIL-USD', 'EOS-USD', 'APE-USD', 'XEM-USD', 'R1-USD', 'ARDX-USD', 'XCN-USD', 'SAND-USD', 'GXE-USD', 'AKT-USD', 'AXS-USD', 'B2M-USD', 'BAND-USD', 'CEL-USD',
           'DAF-USD', 'GMT-USD', 'HNS-USD', 'IOTX-USD', 'KLAY-USD', 'KOK-USD', 'KSM-USD',
           'MKR-USD', 'MNW-USD', 'MYCE-USD', 'ONT-USD', 'PROS-USD', 'RSV-USD', 'SOLVE-USD', 'TRAC-USD', 'TUSD-USD', 'UMA-USD', 'WBTC-USD' ]

# Function to load the data from Yahoo Finance
def load_data(cryptocurrency):
    try:
        data = yf.download(cryptocurrency, period="max")
        data.reset_index(inplace=True)
        data.drop_duplicates(subset='Date', inplace=True)
        data.fillna(0, inplace=True)
        data['date'] = pd.to_datetime(data['Date'])
        return data
    except KeyError:
        print(f"Error: No data available for {cryptocurrency}.")
        return pd.DataFrame()

# Function to calculate risk score
def calculate_risk_score(predicted_price, current_price):
    if predicted_price > current_price:
        return (predicted_price - current_price) / current_price * 100
    else:
        return 0

# Create and train the LSTM model
def create_and_train_model(train_features, train_target):
    model = Sequential()
    model.add(LSTM(100, input_shape=(1, len(feature_columns))))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    model.fit(train_features, train_target, epochs=50, batch_size=128, verbose=2)
    return model

#Define feature_columns outside the functions to make it accessbile globally
feature_columns = ['Open', 'High', 'Low', 'Volume']


# Function to perform single cryptocurrency prediction
def single_crypto_prediction(cryptocurrency, target_date, trained_model=None):
    # Load the data for the selected cryptocurrency
    data = load_data(cryptocurrency)

    if data.empty:
        print(f"Skipping prediction for {cryptocurrency} due to lack of data")
        return None, None

    # Prepare the data for prediction
    target_column = 'Close'
    feature_columns = ['Open', 'High', 'Low', 'Volume']

    target = data[target_column].values
    features = data[feature_columns].values

    target_scaler = MinMaxScaler(feature_range=(0, 1))
    feature_scaler = MinMaxScaler(feature_range=(0, 1))

    scaled_target = target_scaler.fit_transform(target.reshape(-1, 1))
    scaled_features = feature_scaler.fit_transform(features)

    train_size = int(len(data) * 0.8)
    train_target = scaled_target[:train_size]
    train_features = scaled_features[:train_size]
    test_target = scaled_target[train_size:]
    test_features = scaled_features[train_size:]

    train_features = np.reshape(train_features, (train_features.shape[0], 1, train_features.shape[1]))
    test_features = np.reshape(test_features, (test_features.shape[0], 1, test_features.shape[1]))

    if trained_model is None:
        model = create_and_train_model(train_features, train_target)
    else:
        model = trained_model

    current_date = datetime.now().date()  # Convert to datetime.date
    last_known_date = data['date'].max().date()  # Convert to datetime.date
    days_to_predict = (target_date - last_known_date).days

    prediction_dates = [target_date]

    predictions = {}
    risks = {}

    for prediction_date in prediction_dates:
        days_to_predict = (prediction_date - current_date).days
        last_features = test_features[-1:]  # Use the last available features for prediction

        predictions[cryptocurrency] = []
        risks[cryptocurrency] = []

        for _ in range(days_to_predict):
            prediction = model.predict(last_features)
            predicted_price = target_scaler.inverse_transform(prediction)[0][0]
            predictions[cryptocurrency].append(predicted_price)

            # Calculate risk score
            risk_score = calculate_risk_score(predicted_price, target_scaler.inverse_transform(test_target[-1:])[0][0])
            risks[cryptocurrency].append(risk_score)

            # Update the features for the next prediction
            last_features[0][0][0] = prediction[0][0]
            last_features = np.array(last_features)

    # Print the cryptocurrency name and prediction completion message
    print(f"{cryptocurrency} prediction completed.")
    print(f"{cryptocurrency}: Predicted Price: {predictions[cryptocurrency][-1]:.4f}, Risk Score: {risks[cryptocurrency][-1]:.2f}%")

    return predictions, risks

# Function to perform highest profit with 5 cryptos prediction
def highest_profit_prediction(target_date):
    # Dictionary to store predicted prices and risk scores for each cryptocurrency
    predictions = {}
    risks = {}

    for cryptocurrency in tickers:
        cryptocurrency_predictions, cryptocurrency_risks = single_crypto_prediction(cryptocurrency, target_date)

        if cryptocurrency_predictions is not None:
            predictions[cryptocurrency] = cryptocurrency_predictions[cryptocurrency]
            risks[cryptocurrency] = cryptocurrency_risks[cryptocurrency]

    # Sort cryptocurrencies based on risk scores and predicted prices
    sorted_cryptocurrencies = sorted(risks.keys(), key=lambda x: (risks[x][-1], -predictions[x][-1]), reverse=True)

    print(f"Predicted Prices and Risk Scores for {target_date} (Highest Profit with 5 Cryptos):")
    for cryptocurrency in sorted_cryptocurrencies:
        print(f"{cryptocurrency}: Predicted Price: {predictions[cryptocurrency][-1]:.4f}, Risk Score: {risks[cryptocurrency][-1]:.2f}%")

# Function to perform top 5 cryptos with high risk prediction
def top_5_high_risk_prediction(target_date):
    # Dictionary to store predicted prices and risk scores for each cryptocurrency
    predictions = {}
    risks = {}

    for cryptocurrency in tickers:
        cryptocurrency_predictions, cryptocurrency_risks = single_crypto_prediction(cryptocurrency, target_date)

        if cryptocurrency_predictions is not None:
            predictions[cryptocurrency] = cryptocurrency_predictions[cryptocurrency]
            risks[cryptocurrency] = cryptocurrency_risks[cryptocurrency]

    # Sort cryptocurrencies based on risk scores and predicted prices
    sorted_cryptocurrencies = sorted(risks.keys(), key=lambda x: (risks[x][-1], -predictions[x][-1]), reverse=True)

    print(f"Predicted Prices and Risk Scores for {target_date} (Top 5 Cryptos with High Risk):")
    count = 0
    for cryptocurrency in sorted_cryptocurrencies:
        if count == 5:
            break
        print(f"{cryptocurrency}: Predicted Price: {predictions[cryptocurrency][-1]:.4f}, Risk Score: {risks[cryptocurrency][-1]:.2f}%")
        count += 1

# Function to perform top 5 cryptos with medium risk prediction
def top_5_medium_risk_prediction(target_date):
    # Dictionary to store predicted prices and risk scores for each cryptocurrency
    predictions = {}
    risks = {}

    for cryptocurrency in tickers:
        cryptocurrency_predictions, cryptocurrency_risks = single_crypto_prediction(cryptocurrency, target_date)

        if cryptocurrency_predictions is not None:
            predictions[cryptocurrency] = cryptocurrency_predictions[cryptocurrency]
            risks[cryptocurrency] = cryptocurrency_risks[cryptocurrency]

    # Sort cryptocurrencies based on risk scores and predicted prices
    sorted_cryptocurrencies = sorted(risks.keys(), key=lambda x: (risks[x][-1], -predictions[x][-1]), reverse=True)

    print(f"Predicted Prices and Risk Scores for {target_date} (Top 5 Cryptos with Medium Risk):")
    count = 0
    for cryptocurrency in sorted_cryptocurrencies:
        if count == 5:
            break
        print(f"{cryptocurrency}: Predicted Price: {predictions[cryptocurrency][-1]:.4f}, Risk Score: {risks[cryptocurrency][-1]:.2f}%")
        count += 1

# Function to perform top 5 cryptos with low risk prediction
def top_5_low_risk_prediction(target_date):
    # Dictionary to store predicted prices and risk scores for each cryptocurrency
    predictions = {}
    risks = {}

    for cryptocurrency in tickers:
        cryptocurrency_predictions, cryptocurrency_risks = single_crypto_prediction(cryptocurrency, target_date)

        if cryptocurrency_predictions is not None:
            predictions[cryptocurrency] = cryptocurrency_predictions[cryptocurrency]
            risks[cryptocurrency] = cryptocurrency_risks[cryptocurrency]

    # Sort cryptocurrencies based on risk scores and predicted prices
    sorted_cryptocurrencies = sorted(risks.keys(), key=lambda x: (risks[x][-1], -predictions[x][-1]), reverse=False)

    print(f"Predicted Prices and Risk Scores for {target_date} (Top 5 Cryptos with Low Risk):")
    count = 0
    for cryptocurrency in sorted_cryptocurrencies:
        if count == 5:
            break
        print(f"{cryptocurrency}: Predicted Price: {predictions[cryptocurrency][-1]:.4f}, Risk Score: {risks[cryptocurrency][-1]:.2f}%")
        count += 1

# Function to show investment widgets
def show_investment_widgets():
    display(investment_amount_widget, target_date_widget)

# Function to show cryptocurrency dropdown menu and investment widgets
def show_cryptocurrency_dropdown():
    if prediction_mode_dropdown.value == 'Single Crypto Prediction':
        display(cryptocurrency_dropdown, select_button)
    elif prediction_mode_dropdown.value == 'Multi Crypto Prediction':
        show_investment_widgets()

# Create dropdown menu widget for prediction mode
prediction_mode_dropdown = widgets.Dropdown(options=['Single Crypto Prediction',
                                                     'Multi Crypto Prediction'],
                                           description='Prediction Mode:')

# Create "Select" button for single cryptocurrency prediction
select_button = widgets.Button(description='Select')

# Create widgets for user input
investment_amount_widget = widgets.IntSlider(value=1000, min=100, max=10000, step=100, description='Investment Amount ($)')
target_date_widget = widgets.DatePicker(description='Target Date')
cryptocurrency_dropdown = widgets.Dropdown(options=tickers, description='Cryptocurrency:')

# Function to handle the select button click event
def select_button_clicked(b):
    cryptocurrency_dropdown.close()
    show_investment_widgets()

# Register select button click event handler
select_button.on_click(select_button_clicked)

# Create "Predict" button
predict_button = widgets.Button(description='Predict')

# Function to handle the predict button click event
def calculate_total_profit(selected_cryptocurrencies, target_date):
    total_profit = 0
    for cryptocurrency in selected_cryptocurrencies:
        predictions, _ = single_crypto_prediction(cryptocurrency, target_date)
        if predictions is not None and cryptocurrency in predictions:
            predicted_price = predictions[cryptocurrency][-1]
            total_profit += predicted_price

    return total_profit

def predict_button_clicked(b):
    target_date = target_date_widget.value

    prediction_mode = prediction_mode_dropdown.value
    if prediction_mode == 'Single Crypto Prediction':
        cryptocurrency = cryptocurrency_dropdown.value
        single_crypto_prediction(cryptocurrency, target_date)

    elif prediction_mode == 'Multi Crypto Prediction':
        selected_cryptocurrencies = [cryptocurrency_dropdown.value]  # You can select multiple coins here
        top_5_high_risk_prediction(target_date)
        top_5_medium_risk_prediction(target_date)
        top_5_low_risk_prediction(target_date)

        total_profit = calculate_total_profit(selected_cryptocurrencies, target_date)
        print("Total Profit of Selected 5 Cryptos:", total_profit)

# Register predict button click event handler
predict_button.on_click(predict_button_clicked)

# Display the widgets and button
display(prediction_mode_dropdown, select_button, cryptocurrency_dropdown, investment_amount_widget, target_date_widget, predict_button)


Dropdown(description='Prediction Mode:', options=('Single Crypto Prediction', 'Multi Crypto Prediction'), valu…

Button(description='Select', style=ButtonStyle())

Dropdown(description='Cryptocurrency:', options=('BTC-USD', 'PXP-USD', 'XRP-USD', 'ETH-USD', 'XDC-USD', 'MCH-U…

IntSlider(value=1000, description='Investment Amount ($)', max=10000, min=100, step=100)

DatePicker(value=None, description='Target Date')

Button(description='Predict', style=ButtonStyle())

IntSlider(value=1000, description='Investment Amount ($)', max=10000, min=100, step=100)

DatePicker(value=None, description='Target Date')

[*********************100%***********************]  1 of 1 completed
Epoch 1/50
21/21 - 7s - loss: 0.0365 - 7s/epoch - 350ms/step
Epoch 2/50
21/21 - 0s - loss: 0.0139 - 78ms/epoch - 4ms/step
Epoch 3/50
21/21 - 0s - loss: 0.0055 - 71ms/epoch - 3ms/step
Epoch 4/50
21/21 - 0s - loss: 0.0010 - 70ms/epoch - 3ms/step
Epoch 5/50
21/21 - 0s - loss: 1.5236e-04 - 71ms/epoch - 3ms/step
Epoch 6/50
21/21 - 0s - loss: 1.4112e-04 - 79ms/epoch - 4ms/step
Epoch 7/50
21/21 - 0s - loss: 1.3074e-04 - 100ms/epoch - 5ms/step
Epoch 8/50
21/21 - 0s - loss: 1.2461e-04 - 100ms/epoch - 5ms/step
Epoch 9/50
21/21 - 0s - loss: 1.2059e-04 - 98ms/epoch - 5ms/step
Epoch 10/50
21/21 - 0s - loss: 1.1606e-04 - 96ms/epoch - 5ms/step
Epoch 11/50
21/21 - 0s - loss: 1.1143e-04 - 92ms/epoch - 4ms/step
Epoch 12/50
21/21 - 0s - loss: 1.0787e-04 - 91ms/epoch - 4ms/step
Epoch 13/50
21/21 - 0s - loss: 1.0359e-04 - 109ms/epoch - 5ms/step
Epoch 14/50
21/21 - 0s - loss: 9.9474e-05 - 97ms/epoch - 5ms/step
Epoch 15/50
21/21 - 0s - loss